In [ ]:
# ============================================================================
# TRAIN LAUNCHER (Vast.ai)
# ============================================================================
# Assicurati che 'train_multichannel.py' sia nella stessa cartella di questo notebook.

import sys
import os
import torch

# Aggiunge la directory corrente al path per trovare i tuoi script .py
sys.path.append(os.getcwd())

# Importa la funzione di training dal tuo script esterno
try:
    from train_multichannel import train
    print("✅ Modulo 'train_multichannel' importato correttamente.")
except ImportError as e:
    print(f"❌ ERRORE: Non trovo 'train_multichannel.py'. Assicurati che sia nella cartella: {os.getcwd()}")
    raise e

print("=" * 80)
print(" TRAINING MULTI-CHANNEL MODEL")
print("=" * 80)

# ============================================================================
# ⚙️ CONFIGURAZIONE (Modifica qui i parametri senza usare nano!)
# ============================================================================

# Paths
JSON_FILE = r"/workspace/data/PopulationDataset/final_clustered_samples.json"
BASE_DIR = r"/workspace/data/PopulationDataset"

# Model configuration
MODEL_TYPE = 'baseline'        # 'baseline' o 'dual_branch'
PRETRAINED = True              # Usa pesi ImageNet
FREEZE_BACKBONE = False        # False = traino tutto

# Training hyperparameters
NUM_EPOCHS = 15
BATCH_SIZE = 24               # Riduci a 16 o 8 se "CUDA out of memory"
LEARNING_RATE = 1e-4
PATIENCE = 15                 # Early stopping patience

# Output paths
CHECKPOINT_DIR = 'checkpoints_multichannel'
LOG_DIR = 'runs/multichannel_baseline'

# Hardware
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_WORKERS = 4               # Riduci a 0 se hai errori di DataLoader/Multiprocessing

# ============================================================================
# ESECUZIONE
# ============================================================================

print("\nConfigurazione Attuale:")
print(f"  Dataset: {BASE_DIR}")
print(f"  Model: {MODEL_TYPE} | Pretrained: {PRETRAINED}")
print(f"  Device: {DEVICE} | Batch size: {BATCH_SIZE}")
print(f"  Epochs: {NUM_EPOCHS} | LR: {LEARNING_RATE}")

if DEVICE == 'cpu':
    print("\n⚠️ WARNING: Stai usando la CPU! Sarà lentissimo.")

try:
    print("\n" + "=" * 80)
    print(" 🚀 STARTING TRAINING")
    print("=" * 80)
    
    train(
        # Data
        json_file=JSON_FILE,
        base_dir=BASE_DIR,
        
        # Model
        model_type=MODEL_TYPE,
        pretrained=PRETRAINED,
        freeze_backbone=FREEZE_BACKBONE,
        
        # Training
        num_epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        weight_decay=1e-4,
        
        # Early stopping
        patience=PATIENCE,
        
        # Checkpointing
        checkpoint_dir=CHECKPOINT_DIR,
        save_every=5,
        
        # Logging
        log_dir=LOG_DIR,
        
        # System
        num_workers=NUM_WORKERS,
        device=DEVICE,
        resume_from=None  # Esempio: 'checkpoints_multichannel/checkpoint_epoch_5.pth'
    )
    
    print("\n✅ TRAINING COMPLETATO!")
    print(f"Modello migliore salvato in: {CHECKPOINT_DIR}/best_model.pth")

except KeyboardInterrupt:
    print("\n⚠️ Training interrotto manualmente.")
except Exception as e:
    print(f"\n❌ ERRORE: {e}")
    import traceback
    traceback.print_exc()